In [ ]:
#installing necessary dependencies

!pip install azure-storage-blob python-dotenv

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
#importing necessary libraries
import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import os


In [4]:
#Data Extraction

try:
    data = pd.read_csv(r'zipco_transaction.csv')
    print('Data Extracted successfully')
except Exception as e:
    print(f'An error occured: {e}')

Data Extracted successfully


In [5]:
data.head()

,Date,ProductName,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,...,DeliveryTime_min,OrderType,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail,Staff_Name,Staff_Email,DayOfWeek,TotalSales
0,2023-01-01 0:00:00,Vanilla Cake,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,...,30,In Store,William Adams,"9851 David Green\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com,Sunday,25.064608
1,2023-01-01 1:00:00,Red Velvet Cake,1,7.083070,South,Cash,False,Rainy,23.549497,Average,...,33,In Store,Anthony Wiggins,"24682 Holly Stravenue\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net,Sunday,7.083070
2,2023-01-01 2:00:00,Chocolate Cake,5,6.736064,North,Cash,True,Rainy,NaN,Excellent,...,43,Phone Order,Ashley Duke,10184 Washington Trace Apt. 679\nEast Brandist...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net,Sunday,33.680321
3,2023-01-01 3:00:00,Carrot Cake,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,...,32,Online Order,Brandon Taylor,"87194 Jeff Rue\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net,Jessica Stewart,xwilson@example.org,Sunday,14.629647
4,2023-01-01 4:00:00,Pizza Pepperoni,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,...,58,In Store,Brittany Watkins,"850 Julia Groves\nHartview, WI 95954",759-517-8359,petersjoseph@example.net,Cheryl Carpenter,christine96@example.org,Sunday,7.577727


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1005 non-null   object 
 1   ProductName             1005 non-null   object 
 2   Quantity                1005 non-null   int64  
 3   UnitPrice               1005 non-null   float64
 4   StoreLocation           1005 non-null   object 
 5   PaymentType             1005 non-null   object 
 6   PromotionApplied        1005 non-null   bool   
 7   Weather                 1005 non-null   object 
 8   Temperature             904 non-null    float64
 9   StaffPerformanceRating  1005 non-null   object 
 10  CustomerFeedback        905 non-null    object 
 11  DeliveryTime_min        1005 non-null   int64  
 12  OrderType               1005 non-null   object 
 13  CustomerName            1005 non-null   object 
 14  CustomerAddress         1005 non-null   

In [7]:
#Data cleaning and transformation
#Remove duplicates
data.drop_duplicates(inplace=True)

In [8]:
#Handling missing values (filling missing numeric values with mean or median)
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns

for cols in numeric_columns:
    data.fillna({cols: data[cols].mean()}, inplace=True)

In [9]:
#Handling missing values (filling missing strings /object values with 'Unknown')

string_columns = data.select_dtypes(include=['object']).columns

for cols in string_columns:
    data.fillna({cols: 'Unknown'}, inplace=True)

In [10]:
data['Date'] = pd.to_datetime(data['Date'])

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    1000 non-null   datetime64[ns]
 1   ProductName             1000 non-null   object        
 2   Quantity                1000 non-null   int64         
 3   UnitPrice               1000 non-null   float64       
 4   StoreLocation           1000 non-null   object        
 5   PaymentType             1000 non-null   object        
 6   PromotionApplied        1000 non-null   bool          
 7   Weather                 1000 non-null   object        
 8   Temperature             1000 non-null   float64       
 9   StaffPerformanceRating  1000 non-null   object        
 10  CustomerFeedback        1000 non-null   object        
 11  DeliveryTime_min        1000 non-null   int64         
 12  OrderType               1000 non-null   object        

In [12]:
#Implementing the data model by subsetting the data

data.columns

Index(['Date', 'ProductName', 'Quantity', 'UnitPrice', 'StoreLocation',
       'PaymentType', 'PromotionApplied', 'Weather', 'Temperature',
       'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min',
       'OrderType', 'CustomerName', 'CustomerAddress', 'Customer_PhoneNumber',
       'CustomerEmail', 'Staff_Name', 'Staff_Email', 'DayOfWeek',
       'TotalSales'],
      dtype='object')

In [13]:
#Create the product table
products = data[['ProductName']].drop_duplicates().reset_index(drop=True)
products.index.name = 'ProductID'
products = products.reset_index()

In [14]:
products.head()

,ProductID,ProductName
0,0,Vanilla Cake
1,1,Red Velvet Cake
2,2,Chocolate Cake
3,3,Carrot Cake
4,4,Pizza Pepperoni


In [15]:
#Create the customer table
customers = data[['CustomerName', 'CustomerAddress', 'Customer_PhoneNumber','CustomerEmail']].drop_duplicates().reset_index(drop=True)
customers.index.name = 'CustomerID'
customers = customers.reset_index()

In [16]:
customers.head()

,CustomerID,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail
0,0,William Adams,"9851 David Green\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net
1,1,Anthony Wiggins,"24682 Holly Stravenue\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com
2,2,Ashley Duke,10184 Washington Trace Apt. 679\nEast Brandist...,702.520.3286,cooperwilliam@example.com
3,3,Brandon Taylor,"87194 Jeff Rue\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net
4,4,Brittany Watkins,"850 Julia Groves\nHartview, WI 95954",759-517-8359,petersjoseph@example.net


In [17]:
#staff table
staff = data[['Staff_Name', 'Staff_Email']].drop_duplicates().reset_index(drop=True)
staff.index.name = 'staffID'
staff = staff.reset_index()

In [18]:
staff.head()

,staffID,Staff_Name,Staff_Email
0,0,John Bridges,pdavidson@example.com
1,1,Sarah Bentley,ajohnson@example.net
2,2,Connie Cervantes,michele29@example.net
3,3,Jessica Stewart,xwilson@example.org
4,4,Cheryl Carpenter,christine96@example.org


In [19]:
#creating the transaction table
transaction = data.merge(products, on=['ProductName'], how='left')\
                  .merge(customers, on=['CustomerName', 'CustomerAddress', 'Customer_PhoneNumber','CustomerEmail'], how='left')\
                  .merge(staff, on=['Staff_Name', 'Staff_Email'], how='left')

transaction.index.name = 'TransactionID'

transaction = transaction.reset_index() \
              [['TransactionID','ProductID','CustomerID','staffID','Date', 'Quantity', 'UnitPrice', 'StoreLocation',\
              'PaymentType', 'PromotionApplied', 'Weather', 'Temperature',\
              'StaffPerformanceRating', 'CustomerFeedback', 'DeliveryTime_min',\
             'OrderType', 'DayOfWeek','TotalSales']]

In [20]:
transaction.head()

,TransactionID,ProductID,CustomerID,staffID,Date,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,CustomerFeedback,DeliveryTime_min,OrderType,DayOfWeek,TotalSales
0,0,0,0,0,2023-01-01 00:00:00,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,Neutral,30,In Store,Sunday,25.064608
1,1,1,1,1,2023-01-01 01:00:00,1,7.083070,South,Cash,False,Rainy,23.549497,Average,Positive,33,In Store,Sunday,7.083070
2,2,2,2,2,2023-01-01 02:00:00,5,6.736064,North,Cash,True,Rainy,27.154342,Excellent,Unknown,43,Phone Order,Sunday,33.680321
3,3,3,3,3,2023-01-01 03:00:00,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,Positive,32,Online Order,Sunday,14.629647
4,4,4,4,4,2023-01-01 04:00:00,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,Neutral,58,In Store,Sunday,7.577727


In [21]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   TransactionID           1000 non-null   int64         
 1   ProductID               1000 non-null   int64         
 2   CustomerID              1000 non-null   int64         
 3   staffID                 1000 non-null   int64         
 4   Date                    1000 non-null   datetime64[ns]
 5   Quantity                1000 non-null   int64         
 6   UnitPrice               1000 non-null   float64       
 7   StoreLocation           1000 non-null   object        
 8   PaymentType             1000 non-null   object        
 9   PromotionApplied        1000 non-null   bool          
 10  Weather                 1000 non-null   object        
 11  Temperature             1000 non-null   float64       
 12  StaffPerformanceRating  1000 non-null   object   

In [22]:
data.to_csv('clean_data.csv', index='False')

In [23]:
products.to_csv('products.csv', index=False)
customers.to_csv('customers.csv', index=False)
staff.to_csv('staff.csv', index=False)
transaction.to_csv('transaction.csv', index=False)

In [27]:
#Data loading
#Load the environment variables from the .env files
load_dotenv()

connect_str = os.getenv('AZURE_CONNECTION_STRING_VALUE')
container_name = os.getenv('CONTAINER_NAME')

In [31]:
#Create a Blobservice client object
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client(container_name)

# Load data to Azure storage
files = [
    (data, 'rawdata/clean_data.csv'),
    (products, 'cleandata/products.csv'),
    (customers, 'cleandata/customers.csv'),
    (staff, 'cleandata/staff.csv'),
    (transaction, 'cleandata/transaction.csv')
]

for file, blob_name in files:
    blob_client= container_client.get_blob_client(blob_name)
    output = file.to_csv(index=False)
    blob_client.upload_blob(output, overwrite=True)
    print(f'{blob_name} is loaded into Azure Blob storage successfuly')

rawdata/clean_data.csv is loaded into Azure Blob storage successfuly
cleandata/products.csv is loaded into Azure Blob storage successfuly
cleandata/customers.csv is loaded into Azure Blob storage successfuly
cleandata/staff.csv is loaded into Azure Blob storage successfuly
cleandata/transaction.csv is loaded into Azure Blob storage successfuly
